In [38]:
import scipdf
import arxiv
from tinydb import TinyDB, Query
import subprocess

In [39]:
db_path = './db/db.json'
db = TinyDB(db_path)

In [14]:
# with open('../data/reading_list.txt','r'):
#     arxiv_urls = f.

In [15]:
paper_id = '1912.13318'

In [37]:
def get_text_from_dict(article_dict):
    ret = [article_dict['title'],article_dict['abstract']]
    for section in article_dict['sections']:
        ret.append(section['heading'])
        ret.append(section['text'])
    return '\n\n'.join(ret)

In [35]:
def _download_and_upsert_db(db, arxiv_id, overwrite=False):
    doc = {}
    results = arxiv.query(id_list=[paper_id])
    
    try:
        doc['doi'] = results[0]['doi']
    except:
        raise ValueError("""DOI object field must be populated to deposit article to database.""")
    
    doc['title'] = results[0]['title']
    doc['arxiv_id'] = arxiv_id
    doc['authors'] = results[0]['authors']
    outpath = arxiv.arxiv.download(results[0], dirpath='./data/pdfs')
    print(f'Downloaded pdf to {outpath}')
    doc['outpath'] = outpath
    
    article_dict = scipdf.parse_pdf_to_dict(outpath)
    print(f'Parsed pdf at {outpath}')
    
    doc['article_content'] = article_dict
    txt = get_text_from_dict(article_dict)
    doc['text'] = txt
    
    txtpath = path.replace('pdf','txt')
    
    with open(txtpath,'w') as f:
        f.write(txt)
    print(f'Wrote txt to {txtpath}')
    
    q = Query()
    if not overwrite and db.search(q.doi == doc['doi']):
        raise KeyError("DOI already exists in database.")
    else:
        db.upsert(doc, q.doi == doc['doi'])
    pass

In [36]:
_download_and_upsert_db(db,paper_id,overwrite=True)

In [35]:
def _say(txtpath):
    aiffpath = txtpath.replace('txt','aiff')
    cmd = ['say','-f',txtpath,'-o',aiffpath]
    subprocess.run(cmd)